In [1]:
%%javascript
Jupyter.keyboard_manager.command_shortcuts.add_shortcut('=,=', 'jupyter-notebook:restart-kernel-and-run-all-cells');
Jupyter.keyboard_manager.command_shortcuts.add_shortcut('F2,F2', 'jupyter-notebook:restart-kernel');
Jupyter.keyboard_manager.command_shortcuts.add_shortcut('F1,F1', 'jupyter-notebook:run-all-cells-above');

<IPython.core.display.Javascript object>

In [ ]:
import os
import signal
import sys

from IPython.display import display, clear_output
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import torch
from tqdm import tqdm#tqdm_notebook as tqdm


floorplan_vectorization_path = '/home/ovoinov/work/3ddl/vectorization/FloorplanVectorization'

sys.path.append(floorplan_vectorization_path)
from vectran.renderers.cairo import PT_LINE, PT_BEZIER
from vectran.renderers.differentiable_rendering.sigmoids_renderer.renderer import Renderer


os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

#### Utils

In [ ]:
sys.path.append(floorplan_vectorization_path)
from vectran.renderers.cairo import render as original_render,\
                                    render_with_skeleton as original_render_with_skeleton


linecaps = 'butt'  # be careful, synthetic data was rendered with linecaps='square',
                   # but for efficiency purposes the differentiable renderer only supports 'butt',
                   # ¯\_(ツ)_/¯


def render(data, dimensions, data_representation):
    return original_render(data, dimensions, data_representation, linecaps=linecaps, linejoin='miter')


def render_beziers_pt(beziers_batch):
    beziers_batch = beziers_batch.detach().cpu().numpy()
    rasters = np.stack([(1 - render({PT_BEZIER: beziers}, [w, h], data_representation='vahe').astype(np.float32)/255) for beziers in beziers_batch])
    return rasters


def render_beziers_skeleton(beziers_batch, scaling=4):
    beziers_batch = beziers_batch.detach().cpu().numpy() * scaling
    beziers_batch[..., -2] = 0
    return np.stack([original_render_with_skeleton(
        {PT_BEZIER: beziers}, [w*scaling, h*scaling], data_representation='vahe', linecaps=linecaps, linejoin='miter',
        line_color=(0,0,0), line_width=2, node_size=4, control_line_width=1, control_node_size=2
    ) for beziers in beziers_batch])

#### Parameters and renderer

In [ ]:
dtype = torch.float32
device = torch.device('cuda')
w, h = 64, 64

renderer = Renderer([h, w], dtype=dtype, device=device)

#### GT and intial

In [ ]:
#beziers_gt = np.random.rand(4, 2, 9) * [w, h, w, h, w, h, w, h, 5]
beziers_gt = np.random.rand(400, 10, 9) * [w, h, w, h, w, h, w, h, 5]
beziers_gt[..., -1] += 1
raster_np = np.asarray([render({PT_BEZIER: beziers}, [w, h], data_representation='vahe') for beziers in beziers_gt])
rasters_batch = torch.from_numpy(1 - raster_np.astype(np.float32)/255).type(dtype).to(device)
Image.fromarray(np.hstack(raster_np));

In [ ]:
if False:  # random
    initial_vector = np.random.rand(*beziers_gt.shape) * [w, h, w, h, w, h, w, h, 5]
    initial_vector[..., -1] += 1
elif False:  # perturbed
    initial_vector = beziers_gt.copy() + np.random.randn(*beziers_gt.shape)*5
    initial_vector[..., -1] = initial_vector[..., -1].clip(min=1)
elif True:  # translated
    initial_vector = beziers_gt.copy()
    initial_vector[..., [0,2,4,6]] -= 5

_ = np.asarray([render({PT_BEZIER: beziers}, [w, h], data_representation='vahe') for beziers in initial_vector])
Image.fromarray(np.hstack(_));

#### Initialize optimization

In [ ]:
beziers_batch = torch.from_numpy(initial_vector).type(dtype).to(device)
beziers_batch = torch.nn.functional.pad(beziers_batch, [0, 1], value=1)  # append confidence

patches_to_optimize = np.full(beziers_batch.shape[0], True, np.bool)
sigmoid_rat = 2.
min_width = 1/8
min_confidence = .1


def make_optimizer(parameters_iter):
    return torch.optim.Adam(parameters_iter, lr=1e0)
loss_function = torch.nn.MSELoss()

optimize_confidence = False
optimize_width = True


if optimize_confidence:
    if optimize_width:
        beziers_batch.requires_grad = True
        optimizer = make_optimizer([beziers_batch])
        
        def make_beziers_batch_from_parameters():
            return beziers_batch
        def constrain_parameters():
            beziers_batch.data[..., -2].clamp_(min=min_width)
            beziers_batch.data[..., -1].clamp_(min=min_confidence)
            beziers_batch.data[..., 0].clamp_(min=0, max=w)
            beziers_batch.data[..., 6].clamp_(min=0, max=w)
            beziers_batch.data[..., 1].clamp_(min=0, max=h)
            beziers_batch.data[..., 7].clamp_(min=0, max=h)
    else:
        beziers_coordinates_and_confidence = beziers_batch[..., [0,1,2,3,4,5,6,7,9]].clone()
        beziers_coordinates_and_confidence.requires_grad = True
        optimizer = make_optimizer([beziers_coordinates_and_confidence])
        
        def make_beziers_batch_from_parameters():
            return torch.cat([
                beziers_coordinates_and_confidence[..., :8],
                beziers_batch.data[..., -2:-1],
                beziers_coordinates_and_confidence[..., -1:]
            ], dim=-1)
        def constrain_parameters():
            beziers_coordinates_and_confidence.data[..., -1].clamp_(min=min_confidence)
            beziers_coordinates_and_confidence.data[..., 0].clamp_(min=0, max=w)
            beziers_coordinates_and_confidence.data[..., 6].clamp_(min=0, max=w)
            beziers_coordinates_and_confidence.data[..., 1].clamp_(min=0, max=h)
            beziers_coordinates_and_confidence.data[..., 7].clamp_(min=0, max=h)
else:
    if optimize_width:
        beziers_coordinates_and_width = beziers_batch[..., :9].clone()
        beziers_coordinates_and_width.requires_grad = True
        optimizer = make_optimizer([beziers_coordinates_and_width])
        
        def make_beziers_batch_from_parameters():
            return torch.cat([beziers_coordinates_and_width, beziers_batch.data[..., -1:]], dim=-1)
        def constrain_parameters():
            beziers_coordinates_and_width.data[..., -1].clamp_(min=min_width)
            beziers_coordinates_and_width.data[..., 0].clamp_(min=0, max=w)
            beziers_coordinates_and_width.data[..., 6].clamp_(min=0, max=w)
            beziers_coordinates_and_width.data[..., 1].clamp_(min=0, max=h)
            beziers_coordinates_and_width.data[..., 7].clamp_(min=0, max=h)
    else:
        beziers_coordinates = beziers_batch[..., :8].clone()
        beziers_coordinates.requires_grad = True
        optimizer = make_optimizer([beziers_coordinates])
        
        def make_beziers_batch_from_parameters():
            return torch.cat([beziers_coordinates, beziers_batch.data[..., -2:]], dim=-1)
        def constrain_parameters():
            beziers_coordinates.data[..., 0].clamp_(min=0, max=w)
            beziers_coordinates.data[..., 6].clamp_(min=0, max=w)
            beziers_coordinates.data[..., 1].clamp_(min=0, max=h)
            beziers_coordinates.data[..., 7].clamp_(min=0, max=h)

#### Initialize plotting

In [ ]:
iters_n = 10000
plotting = True
if len(beziers_batch) > 20:
    plotting = False

energies = np.full(iters_n, np.inf)
if plotting:
    ims = []
    fig, axes = plt.subplots(1, 7, figsize=[4 * 7, 4 * len(beziers_batch)])
    initial_pred_ax, initial_skeleton_ax, refined_ax, refined_skeleton_ax, rasterization_ax, gt_ax, dif_ax = axes
    
    initial_pred_ax.set_xlabel('Initial prediction', fontsize=12)
    initial_skeleton_ax.set_xlabel('Initial skeleton', fontsize=12)
    refined_ax.set_xlabel('Refined prediction', fontsize=12)
    refined_skeleton_ax.set_xlabel('Refined skeleton', fontsize=12)
    rasterization_ax.set_xlabel('Differentiable rasterization', fontsize=12)
    gt_ax.set_xlabel('GT', fontsize=12)
    dif_ax.set_xlabel('Difference', fontsize=12)
    
    for ax in axes:
        ax.xaxis.set_label_position('top')
        ax.get_xaxis().set_ticks([])
        ax.get_yaxis().set_ticks([])    
    fig.subplots_adjust(wspace=0, hspace=0)

## Optimize

In [ ]:
vector_rendering = render_beziers_pt(beziers_batch)
if plotting:
    im_gt = rasters_batch.cpu().numpy()
    im_dif = im_gt - vector_rendering
    im_initial_pred = vector_rendering
    im_refined = im_initial_pred
    im_initial_skeleton = render_beziers_skeleton(beziers_batch)
    im_refined_skeleton = im_initial_skeleton
    im_rasterization = renderer.render(beziers_batch.data, sigmoid_rate=sigmoid_rat).detach().cpu().numpy()
    
    initial_pred_plot = initial_pred_ax.imshow(np.vstack(im_initial_pred), vmin=0, vmax=1, cmap='gray_r')
    refined_plot = refined_ax.imshow(np.vstack(im_refined), vmin=0, vmax=1, cmap='gray_r')
    gt_plot = gt_ax.imshow(np.vstack(im_gt), vmin=0, vmax=1, cmap='gray_r')
    dif_plot = dif_ax.imshow(np.vstack(im_dif), vmin=-1, vmax=1, cmap='gray_r')
    initial_skeleton_plot = initial_skeleton_ax.imshow(np.vstack(im_initial_skeleton))
    refined_skeleton_plot = refined_skeleton_ax.imshow(np.vstack(im_refined_skeleton))
    rasterization_plot = rasterization_ax.imshow(np.vstack(im_rasterization), vmin=0, vmax=1, cmap='gray_r')


its_time_to_stop = [False]
def plotting_sigint(*args):
    its_time_to_stop[0] = True
    
for i in tqdm(range(iters_n)):
#for i in range(iters_n):
    try:
        beziers_batch = make_beziers_batch_from_parameters()
        rasterizations_batch = renderer.render(beziers_batch, sigmoid_rate=sigmoid_rat)
        loss = loss_function(rasterizations_batch, rasters_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        constrain_parameters()
        energies[i] = loss.item()
    except KeyboardInterrupt:
        its_time_to_stop[0] = True
    # Prepare the results for logging
    sigint = signal.signal(signal.SIGINT, plotting_sigint)
    if (i % 1 == 0) or its_time_to_stop[0]:
        vector_rendering[patches_to_optimize] = render_beziers_pt(beziers_batch[patches_to_optimize])
        im = rasters_batch[patches_to_optimize].cpu().numpy() - vector_rendering[patches_to_optimize]
        if plotting:
            im_refined[patches_to_optimize] = vector_rendering[patches_to_optimize]
            im_dif[patches_to_optimize] = im
            im_refined_skeleton[patches_to_optimize] = render_beziers_skeleton(beziers_batch[patches_to_optimize])
            im_rasterization[patches_to_optimize] = rasterizations_batch.detach().cpu().numpy()
            refined_plot.set_array(np.vstack(im_refined))
            dif_plot.set_array(np.vstack(im_dif))
            refined_skeleton_plot.set_array(np.vstack(im_refined_skeleton))
            rasterization_plot.set_array(np.vstack(im_rasterization))
            clear_output(wait=True)
            display(fig)
            # Optionally, save the images for further export
            ## ims.append([im_refined.copy(), im_dif.copy(), im_refined_skeleton.copy(), im_rasterization.copy()])
    signal.signal(signal.SIGINT, sigint)
    if its_time_to_stop[0]:
        break

#### Export results